# SAMPLE()

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession.builder
    .appName("example-sample-resample")
    .getOrCreate()
)

## Using fraction to get a random sample in PySpark

By using fraction between 0 to 1, it returns the approximate number of the fraction of the dataset. For example, 0.1 returns 10% of the rows. However, this does not guarantee it returns the exact 10% of the records

In [3]:
# My DataFrame has 100 records and I wanted to get 6% sample records which are 6 but the sample() function returned 7 records. 
# This proves the sample function doesn’t return the exact fraction specified.
df = spark.range(100)
print(df.sample(0.06).collect())

[Row(id=30), Row(id=32), Row(id=65), Row(id=76), Row(id=91), Row(id=97)]


## Using seed to reproduce the same Samples in PySpark

Every time you run a sample() function it returns a different set of sampling records, however sometimes during the development and testing phase you may need to regenerate the same sample every time as you need to compare the results from your previous run. To get consistent same random sampling uses the same slice value for every run. Change slice value to get different results.

In [4]:
print(df.sample(0.1,123).collect())

print(df.sample(0.1,123).collect())

print(df.sample(0.1,456).collect())


[Row(id=35), Row(id=38), Row(id=41), Row(id=45), Row(id=71), Row(id=84), Row(id=87), Row(id=99)]
[Row(id=35), Row(id=38), Row(id=41), Row(id=45), Row(id=71), Row(id=84), Row(id=87), Row(id=99)]
[Row(id=22), Row(id=33), Row(id=35), Row(id=41), Row(id=53), Row(id=80), Row(id=83), Row(id=87), Row(id=92)]


## Sample withReplacement (May contain duplicates)

Some times you may need to get a random sample with repeated values. By using the value true, results in repeated values.

In [5]:
print(df.sample(True, 0.3, 123).collect()) # with Duplicates


[Row(id=0), Row(id=5), Row(id=9), Row(id=11), Row(id=13), Row(id=16), Row(id=17), Row(id=26), Row(id=26), Row(id=37), Row(id=41), Row(id=45), Row(id=49), Row(id=50), Row(id=50), Row(id=57), Row(id=58), Row(id=58), Row(id=65), Row(id=66), Row(id=71), Row(id=74), Row(id=77), Row(id=80), Row(id=81), Row(id=82), Row(id=84), Row(id=88), Row(id=90), Row(id=91), Row(id=91), Row(id=92), Row(id=94), Row(id=96)]


In [6]:
print(df.sample(0.3, 123).collect()) # No duplicates


[Row(id=0), Row(id=4), Row(id=12), Row(id=15), Row(id=19), Row(id=21), Row(id=23), Row(id=24), Row(id=25), Row(id=28), Row(id=29), Row(id=34), Row(id=35), Row(id=36), Row(id=38), Row(id=41), Row(id=45), Row(id=47), Row(id=50), Row(id=52), Row(id=59), Row(id=63), Row(id=65), Row(id=71), Row(id=82), Row(id=84), Row(id=87), Row(id=94), Row(id=99)]


## Stratified sampling in PySpark

You can get Stratified sampling in PySpark without replacement by using sampleBy() method. It returns a sampling fraction for each stratum. If a stratum is not specified, it takes zero as the default.

In [7]:
df2 = df.select((df.id % 3).alias("key"))
print(df2.sampleBy("key", {0: 0.1, 1: 0.2},0).collect())


[Row(key=0), Row(key=0), Row(key=1), Row(key=1), Row(key=0), Row(key=1), Row(key=0), Row(key=1), Row(key=0), Row(key=0), Row(key=1), Row(key=1), Row(key=0)]


Which of the following code blocks returns about 150 randomly selected rows from the 1000-row DataFrame transactionsDf, assuming that any row can appear more than once in the returned DataFrame?

- `transactionsDf.resample(0.15, False, 3142)`
- `transactionsDf.sample(0.15, False, 3142)`
- `transactionsDf.sample(0.15)`
- `transactionsDf.sample(0.85, 8429)`
- `transactionsDf.sample(True, 0.15, 8261)`

In [2]:
transactionsDf = spark.range(1000)

In [22]:
transactionsDf.sample(0.15, False, 3142).count()

166

In [23]:
transactionsDf.sample(0.15).count()

154

In [24]:
transactionsDf.sample(0.85, 8429).count()

859

In [25]:
transactionsDf.sample(True, 0.15, 8261).count()

150

The code block shown below should return only the average prediction error (column predError) of a random subset, without replacement, of approximately 15% of rows in DataFrame transactionsDf. Choose the answer that correctly fills the blanks in the code block to accomplish this.
>
- `transactionsDf.__1__(__2__, __3__).__4__(avg(‘predError’))`
>
- `1. sample 2. True 3. 0.15 4. filter`
- `1. sample 2. False 3. 0.15 4. select`
- `1. sample 2. 0.85 3. False 4. select`
- `1. fraction 2. 0.15 3. True 4. where`
- `1. fraction 2. False 3. 0.85 4. select`

In [4]:
data = [(1, 3, 4, 25, 1, None, 1587915332),
         (2, 6, 7, 2, 2, None, 1586815312),
         (3, 3, None, 25, 3, None, 1585824821),
         (4, None, None, 3, 2, None, 1583244275),
         (5, None, None, None, 2, None, 1575285427),
         (6, 3, 2, 25, 2, None, 1572733275)]

schema = StructType([StructField('transactionId', IntegerType(), True),
                     StructField('predError', IntegerType(), True),
                     StructField('value', IntegerType(), True),
                     StructField('storeId', IntegerType(), True),
                     StructField('productId', IntegerType(), True),
                     StructField('f', IntegerType(), True),
                     StructField('transactionDate', LongType(), True)])

transactionsDf = spark.createDataFrame(data=data, schema=schema)

In [7]:
transactionsDf.sample(False, 0.15).select(avg('predError')).show()

+--------------+
|avg(predError)|
+--------------+
|          null|
+--------------+



Which of the following code blocks returns approximately 1000 rows, some of them potentially being duplicates, from the 2000-row DataFrame transactionsDf that only has unique rows?
>
- `transactionsDf.sample(True, 0.5)`
- `transactionsDf.take(1000).distinct()`
- `transactionsDf.sample(False, 0.5)`
- `transactionsDf.take(1000)`
- `transactionsDf.sample(True, 0.5, force=True)`

In [13]:
df = spark.range(2000)

In [14]:
df.sample(True, 0.5).count()

1028

In [12]:
df.sample(False, 0.5).count()


1004

In [ ]:
# AttributeError: 'list' object has no attribute 'distinct'
df.take(1000).distinct()

# TypeError: sample() got an unexpected keyword argument 'force'
df.sample(True, 0.5, force=True)